### Self-consistent field (SCF)

Now that we have all the necessary components at hand, we'll construct a full self-consistent cycle. We start by creating the initial guess for the density matrix using the core Hamiltonian.

In [2]:
from pyscf import gto, scf, mp
from ase.units import Bohr
import numpy as np

m = gto.Mole()
m.build(atom=f"H 0 0 0; H {1.4*Bohr} 0 0", basis="3-21g", spin=0, charge=0)
Tmat = m.intor("int1e_kin") # Integral matrix for kinetic energy
Smat = m.intor("int1e_ovlp") # Integral matrix for overlap integrals
Vmat = m.intor("int1e_nuc") # Integral matrix for electron-nucleus integrals 
Vee = m.intor("int2e") # Integral matrix for electron-electron integrals
nelec = m.nelectron # Number of electrons

# Symmetric orthogonalization
s, U = np.linalg.eigh(Smat)
X = U / s**0.5

# Construct the core Hamiltonian
Hcore = Tmat + Vmat
Hcore_ = np.dot(X.T, Hcore.dot(X)) # Transform from AO to MO basis
mo_eigs, mo_vecs = np.linalg.eigh(Hcore_) # Diagonalize the Hcore in MO basis

Cvec = np.dot(X, mo_vecs) # Transform the MO coefficients from MO to AO basis
Dmat = np.zeros((Smat.shape)) # Initialize the density matrix
for i in range(nelec//2): # Compute the density matrix in the AO basis. Since we use restricted HF, all spins are paired and only nelec/2 orbitals are considered
    Dmat += np.einsum('i,j->ij', Cvec[:, i], Cvec[:, i].T)

We'll define the cycle a loop. Each cycle we'll construct the Fockian matrix using the density matrix from the previous cycle in AO basis (or, the initial guess for the first iteration). We then transform the Fockian into the MO basis and diagonalize it. This will result in new set of molecular orbitals and orbital energies. We'll store the old density matrix and compare the matrix norm between the new and the old density matrices. If the difference is sufficiently small, we'll determine that the SCF cycle has converged. Real quantum chemistry codes use multiple indicators to check for convergence (density, energy, gradient etc.). We should also set a maximum number of SCF iterations so that we do not end up in an infinite loop.

In [3]:
iterstep = 0
maxiter = 50
conv_tol = 1e-6
print(f"{'Iteration':>10} {'Total Energy':>15} {'Dmat Difference':>18}")
while iterstep < maxiter:
    Jmat = np.einsum('kl, ijkl->ij', Dmat, Vee, optimize=True) # Compute the Coulomb integrals
    Kmat = np.einsum('kl, ilkj->ij', Dmat, Vee, optimize=True) # Compute the exchange integrals
        
    Fmat = Hcore + 2*Jmat - Kmat # Construct the full Fock matrix
    e_elec = np.trace(np.dot(2*Hcore + 2*Jmat - Kmat, Dmat))
    # Transform to orthonormal MO basis
    Fmat_ = np.dot(X.T, Fmat.dot(X))
    Dmat_old = Dmat.copy() # Store old density matrix
        
    mo_eigs, mo_vecs = np.linalg.eigh(Fmat_)
    Cvec = np.dot(X, mo_vecs)

    Dmat *= 0
    for i in range(nelec//2): # RHF
        Dmat += np.einsum('i,j->ij', Cvec[:, i], Cvec[:, i].T)
    conv = np.linalg.norm(Dmat-Dmat_old)
    iterstep += 1
    print(f"{iterstep:>10} {e_elec:>15.8f} {conv:>18.3e}")
    if conv < conv_tol:
        break

 Iteration    Total Energy    Dmat Difference
         1     -1.78732579          1.533e-01
         2     -1.83611232          2.198e-02
         3     -1.83719589          3.175e-03
         4     -1.83721860          4.576e-04
         5     -1.83721907          6.595e-05
         6     -1.83721908          9.502e-06
         7     -1.83721908          1.369e-06
         8     -1.83721908          1.973e-07


Yay! Our little SCF program converged! We see a monotonic convergence in both energy and density. For consistency, let's compare our results with that of PySCF using the same initial guess.

In [4]:
hf = m.RHF()
hf.verbose = 5
hf.max_cycle = 50
hf.init_guess = 'hcore'
hf.run()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = hcore
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/weckman/Python/tmpy0g9hpmy
max_memory 4000 MB (current use 147 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from hcore.
  HOMO = -1.26830366856786  LUMO = -0.604330213847378
  mo_energy =
[-1.26830367 -0.60433021 -0.06295639  0.36737513]
E1 = -2.5366073371357185  E_coul = 0.7492815459509367
init E= -1.07304007641832
cond(S) = 25.587367648833535
    CPU time for initialize scf      0.17 sec, wall time      0.01 sec
  HOMO = -0.54010068175068  LUMO = 0.298013288147918
  mo_energy =
[-0.54010068  0.29801329  0.94830472  1.59943342]
E1 = -2.5024674666487083  E_coul = 0.6663551424066831
cycle= 1 E= -1.12182660947557  delta_E= -0.0488  |g|= 0

The difference between the PySCF and our results is of course the nuclear repulsion energy, often included as part of the electronic energy. If we include this to our total energy, our results agree nicely.

In [5]:
print(e_elec + m.get_enuc())

-1.1229333636214585
